In [ ]:
# from rm_lite.tools_1d import rmclean
import numpy as np

from rm_lite.tools_1d import rmsynth
from rm_lite.utils import clean as clean_utils
from rm_lite.utils.synthesis import freq_to_lambda2, make_phi_array, inverse_rmsynth_nufft, make_double_phi_array, get_rmsf_nufft

import astropy.units as u
from scipy.ndimage import convolve

import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 150

In [ ]:
def delta_func(x, x_0):
    y = np.zeros_like(x)
    y[np.argmin(np.abs(x - x_0))] = 1
    return y


def slab_func(x, x0, w):
    y = np.zeros_like(x)
    y[np.abs(x - x0) < w / 2] = 1
    return y

def gaussian_func(x, x0, w):
    y = np.exp(-2 * (x - x0)**2 / w**2)
    return y

def faraday_slab_spectrum(
    freq_arr_hz: np.ndarray,
    frac_pol: float,
    psi0_deg: float,
    rm_radm2: float,
    delta_rm_radm2: float,
) -> np.ndarray:
    """Create a simple Faraday spectrum with a single component.

    Args:
        freq_arr_hz (np.ndarray): Frequency array in Hz
        frac_pol (float): Fractional polarization
        psi0_deg (float): Initial polarization angle in degrees
        rm_radm2 (float): RM in rad/m^2

    Returns:
        np.ndarray: Complex polarization spectrum
    """
    lambda_sq_arr_m2 = freq_to_lambda2(freq_arr_hz)

    complex_polarization = (
        frac_pol
        * np.exp(2j * (np.deg2rad(psi0_deg) + rm_radm2 * lambda_sq_arr_m2))
        * (
            np.sin(delta_rm_radm2 * lambda_sq_arr_m2)
            / (delta_rm_radm2 * lambda_sq_arr_m2)
        )
    )

    return complex_polarization


def faraday_gaussian_spectrum(
    freq_arr_hz: np.ndarray,
    frac_pol: float,
    psi0_deg: float,
    rm_radm2: float,
    sigma_rm_radm2: float,
):
    lambda_sq_arr_m2 = freq_to_lambda2(freq_arr_hz)
    rm_term = np.exp(2j * (np.deg2rad(psi0_deg) + rm_radm2 * lambda_sq_arr_m2)) 
    depol_term = np.exp(-2.0 * sigma_rm_radm2**2 * lambda_sq_arr_m2**2)
    complex_polarization = frac_pol * rm_term * depol_term
    return complex_polarization



In [ ]:
bw_low = 288
bw_mid = 144
bw_high = 288
low = np.linspace(943.5 - bw_low / 2, 943.5 + bw_low / 2, 36) * u.MHz
mid = np.linspace(1367.5 - bw_mid / 2, 1367.5 + bw_mid / 2, 9) * u.MHz
high = np.linspace(1655.5 - bw_high / 2, 1655.5 + bw_high / 2, 9) * u.MHz
freqs = np.concatenate([low, mid, high])
# freqs = np.linspace(943.5 - 288 / 2, 1655.5 + 288 / 2, 100) * u.MHz

# freq_hz = np.arange(744, 1032, 8) * 1e6
freq_hz = freqs.to(u.Hz).value

In [ ]:
phi_arr_radm2 = make_phi_array(phi_max_radm2=1000, d_phi_radm2=1)
delta_rm_radm2 = 30
rm = 0

complex_data_noiseless = faraday_slab_spectrum(
    freq_arr_hz=freq_hz,
    frac_pol=1,
    psi0_deg=0,
    rm_radm2=rm,
    delta_rm_radm2=delta_rm_radm2,
)
input_model_fdf = slab_func(phi_arr_radm2, rm, delta_rm_radm2) * (
    1 + 0j
)
input_model_fdf = input_model_fdf / np.sum(np.abs(input_model_fdf))

lambda_sq_arr_m2 = freq_to_lambda2(freq_hz)
lam_sq_0_m2 = np.nanmean(lambda_sq_arr_m2)

lambda_sq_model_arr = np.linspace(0, 0.5, 1000)
input_model_spec = inverse_rmsynth_nufft(
    fdf_q_array=input_model_fdf.real,
    fdf_u_array=input_model_fdf.imag,
    lambda_sq_arr_m2=lambda_sq_model_arr,
    phi_arr_radm2=phi_arr_radm2,
    lam_sq_0_m2=0,
)

complex_data = complex_data_noiseless
fdf_dirty = rmsynth.rmsynth_nufft(
    stokes_q_array=complex_data.real,
    stokes_u_array=complex_data.imag,
    lambda_sq_arr_m2=lambda_sq_arr_m2,
    phi_arr_radm2=phi_arr_radm2,
    lam_sq_0_m2=lam_sq_0_m2,
    weight_array=np.ones_like(complex_data.real),
)
rmsf_cube, phi_double_arr_radm2, fwhm_rmsf_arr, fit_status_array = get_rmsf_nufft(
    lambda_sq_arr_m2=lambda_sq_arr_m2,
    phi_arr_radm2=phi_arr_radm2,
    weight_array=np.ones_like(complex_data.real),
    lam_sq_0_m2=lam_sq_0_m2,
    do_fit_rmsf=True,
)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8), sharey=False, sharex=False)
ax1, ax2, ax3, ax4 = axs.flatten()
ax1.plot(phi_arr_radm2, input_model_fdf.real, "-", color="tab:blue", label="Stokes Q")
ax1.plot(phi_arr_radm2, input_model_fdf.imag, "-", color="tab:red", label="Stokes U")
ax2.plot(lambda_sq_model_arr, input_model_spec.real, "-", color="tab:blue", label="Stokes Q")
ax2.plot(lambda_sq_model_arr, input_model_spec.imag, "-", color="tab:red", label="Stokes U")
ax2.plot(lambda_sq_model_arr, np.abs(input_model_spec), "-", color="k", label="PI")
ax1.set(xlabel="$\phi / $(rad m$^{-2}$)", ylabel="Flux Density")
ax2.set(xlabel="$\lambda^2 / $(m$^2$)", ylabel="Flux Density")

ax3.plot(phi_arr_radm2, np.abs(fdf_dirty), "-",  lw=1, alpha=1, color="k", label="Dirty")
ax3.plot(phi_arr_radm2, fdf_dirty.real, "-",  lw=1, alpha=1, color="tab:blue", label="Stokes Q")
ax3.plot(phi_arr_radm2, fdf_dirty.imag, "-",  lw=1, alpha=1, color="tab:red", label="Stokes U")

ax3.legend()
ax3.set(xlabel="$\phi / $(rad m$^{-2}$)", ylabel="Flux Density")

ax4.plot(lambda_sq_arr_m2, complex_data.real, ".", color="tab:blue", label="Stokes Q")
ax4.plot(lambda_sq_arr_m2, complex_data.imag, ".", color="tab:red", label="Stokes U")
ax4.plot(lambda_sq_arr_m2, np.abs(complex_data), ".", color="k", label="PI")
ax4.set(xlabel="$\lambda^2 / $(m$^2$)", ylabel="Flux Density", ylim=(-1.1, 1.1))

In [ ]:
resid_fdf_spectrum = fdf_dirty.copy()
resid_fdf_spectrum_mask = np.ma.masked_array(resid_fdf_spectrum)
phi_arr_radm2 = phi_arr_radm2
phi_double_arr_radm2 = make_double_phi_array(phi_arr_radm2)
rmsf_spectrum = rmsf_cube.copy()
rmsf_fwhm = float(fwhm_rmsf_arr)


In [ ]:
minor_loop_res = clean_utils.minor_loop(
    resid_fdf_spectrum_mask=resid_fdf_spectrum_mask,
    phi_arr_radm2=phi_arr_radm2,
    phi_double_arr_radm2=phi_double_arr_radm2,
    rmsf_spectrum=rmsf_spectrum,
    rmsf_fwhm=rmsf_fwhm,
    max_iter=1,
    threshold=0,
    mask=0,
    gain=1
)
fig, ax = plt.subplots()
ax.plot(
    phi_arr_radm2,
    np.abs(minor_loop_res.resid_fdf_spectrum),
    "-",
    color="k",
    label="Residual",
    lw=1
)
ax.plot(
    phi_arr_radm2,
    np.abs(resid_fdf_spectrum),
    "-",
    color="k",
    alpha=0.5,
    lw=1,
)

In [ ]:
2.0 * np.sqrt(2.0 * np.log(2.0))

In [ ]:
from rm_lite.utils.fitting import fit_rmsf, fwhm_to_sigma, sigma_to_fwhm


resid_fdf_spectrum = fdf_dirty.copy()
resid_fdf_spectrum_mask = np.ma.masked_array(resid_fdf_spectrum)
phi_arr_radm2 = phi_arr_radm2
phi_double_arr_radm2 = make_double_phi_array(phi_arr_radm2)
rmsf_spectrum = rmsf_cube.copy()
rmsf_fwhm = float(fwhm_rmsf_arr)


conv_kernel = slab_func(phi_double_arr_radm2, 0, delta_rm_radm2)
conv_kernel /= np.sum(np.abs(conv_kernel))


fdf_abs_conv = convolve(
    np.abs(resid_fdf_spectrum),
    conv_kernel,
    mode="reflect"
)


resid_fdf_spectrum_conv = fdf_abs_conv * np.exp(1j * np.angle(resid_fdf_spectrum))
resid_fdf_spectrum_conv_mask = np.ma.masked_array(resid_fdf_spectrum_conv)

fwhm_guess = sigma_to_fwhm(np.hypot(fwhm_to_sigma(rmsf_fwhm), fwhm_to_sigma(delta_rm_radm2)))

conv_kernel = slab_func(phi_double_arr_radm2, 0, delta_rm_radm2)
conv_kernel /= np.max(np.abs(conv_kernel))
rmsf_abs_conv = convolve(
    np.abs(rmsf_spectrum),
    conv_kernel,
    mode="reflect"
)

rmsf_abs_conv /= np.max(np.abs(rmsf_abs_conv))
rmsf_spectrum_conv = rmsf_abs_conv * np.exp(1j * np.angle(rmsf_spectrum))

rmsf_fwhm_conv = fit_rmsf(
    rmsf_to_fit_array=np.abs(rmsf_spectrum_conv),
    phi_double_arr_radm2=phi_double_arr_radm2,
    fwhm_rmsf_radm2=fwhm_guess
)

fig, ax = plt.subplots()
ax.plot(phi_double_arr_radm2, conv_kernel / np.sum(np.abs(conv_kernel)), "-", lw=1, label="Convolution Kernel")
ax.plot(phi_arr_radm2, np.abs(resid_fdf_spectrum), "-", lw=1, label="Residual FDF")
ax.plot(phi_arr_radm2, np.abs(resid_fdf_spectrum_conv), "-", lw=1, label="Convolved Residual FDF")
ax.set(xlim=(-100,100), xlabel="$\phi$ / (rad/m$^2$)", ylabel="Flux Density / (units / RMSF)")
ax.legend()

fig, ax = plt.subplots()
ax.plot(phi_double_arr_radm2, np.abs(rmsf_spectrum), "-", lw=1, label="RMSF")
ax.plot(phi_double_arr_radm2, np.abs(rmsf_spectrum_conv), "-", lw=1, label="Convolved RMSF")
ax.plot(phi_double_arr_radm2, conv_kernel, "-", lw=1, label="Convolution Kernel")
ax.legend()
ax.set(xlim=(-100,100), xlabel="$\phi$ / (rad/m$^2$)", ylabel="RMSF")

fig, ax = plt.subplots()
ax.plot(phi_arr_radm2, np.abs(resid_fdf_spectrum) / np.abs(resid_fdf_spectrum).max(), "-", lw=1, label="Residual FDF")
ax.plot(phi_arr_radm2, np.abs(resid_fdf_spectrum_conv) / np.abs(resid_fdf_spectrum_conv).max(), "-", lw=1, label="Convolved Residual FDF")
ax.plot(phi_double_arr_radm2, np.abs(rmsf_spectrum_conv), "-", lw=1, label="Convolved RMSF")
ax.plot(phi_double_arr_radm2, conv_kernel, "-", lw=1, label="Convolution Kernel")
ax.set(xlim=(-100,100), xlabel="$\phi$ / (rad/m$^2$)", ylabel="Normalised")
ax.legend()

In [ ]:
from rm_lite.utils.fitting import fit_rmsf, fwhm_to_sigma, sigma_to_fwhm


resid_fdf_spectrum = fdf_dirty.copy()
resid_fdf_spectrum_mask = np.ma.masked_array(resid_fdf_spectrum)
phi_arr_radm2 = phi_arr_radm2
phi_double_arr_radm2 = make_double_phi_array(phi_arr_radm2)
rmsf_spectrum = rmsf_cube.copy()
rmsf_fwhm = float(fwhm_rmsf_arr)


conv_kernel = slab_func(phi_double_arr_radm2, 0, delta_rm_radm2)
conv_kernel /= np.sum(np.abs(conv_kernel))


fdf_r_conv = convolve(
    resid_fdf_spectrum.real,
    conv_kernel,
    mode="reflect"
)
fdf_i_conv = convolve(
    resid_fdf_spectrum.imag,
    conv_kernel,
    mode="reflect"
)

resid_fdf_spectrum_conv = fdf_r_conv + 1j * fdf_i_conv
resid_fdf_spectrum_conv_mask = np.ma.masked_array(resid_fdf_spectrum_conv)

fwhm_guess = sigma_to_fwhm(np.hypot(fwhm_to_sigma(rmsf_fwhm), fwhm_to_sigma(delta_rm_radm2)))


rmsf_r_conv = convolve(
    rmsf_spectrum.real,
    conv_kernel,
    mode="reflect"
)
rmsf_i_conv = convolve(
    rmsf_spectrum.imag,
    conv_kernel,
    mode="reflect"
)


rmsf_spectrum_conv = rmsf_r_conv + 1j * rmsf_i_conv
rmsf_spectrum_conv /= np.abs(rmsf_spectrum_conv).max()

rmsf_fwhm_conv = fit_rmsf(
    rmsf_to_fit_array=np.abs(rmsf_spectrum_conv),
    phi_double_arr_radm2=phi_double_arr_radm2,
    fwhm_rmsf_radm2=fwhm_guess
)

fig, ax = plt.subplots()
ax.plot(phi_double_arr_radm2, conv_kernel / np.sum(np.abs(conv_kernel)), "-", lw=1, label="Convolution Kernel")
ax.plot(phi_arr_radm2, np.abs(resid_fdf_spectrum), "-", lw=1, label="Residual FDF")
ax.plot(phi_arr_radm2, np.abs(resid_fdf_spectrum_conv), "-", lw=1, label="Convolved Residual FDF")
ax.set(xlim=(-100,100), xlabel="$\phi$ / (rad/m$^2$)", ylabel="Flux Density / (units / RMSF)")
ax.legend()

fig, ax = plt.subplots()
ax.plot(phi_double_arr_radm2, np.abs(rmsf_spectrum), "-", lw=1, label="RMSF")
ax.plot(phi_double_arr_radm2, np.abs(rmsf_spectrum_conv), "-", lw=1, label="Convolved RMSF")
ax.plot(phi_double_arr_radm2, conv_kernel, "-", lw=1, label="Convolution Kernel")
ax.legend()
ax.set(xlim=(-100,100), xlabel="$\phi$ / (rad/m$^2$)", ylabel="RMSF")

fig, ax = plt.subplots()
ax.plot(phi_arr_radm2, np.abs(resid_fdf_spectrum) / np.abs(resid_fdf_spectrum).max(), "-", lw=1, label="Residual FDF")
ax.plot(phi_arr_radm2, np.abs(resid_fdf_spectrum_conv) / np.abs(resid_fdf_spectrum_conv).max(), "-", lw=1, label="Convolved Residual FDF")
ax.plot(phi_double_arr_radm2, np.abs(rmsf_spectrum_conv), "-", lw=1, label="Convolved RMSF")
ax.plot(phi_double_arr_radm2, conv_kernel, "-", lw=1, label="Convolution Kernel")
ax.set(xlim=(-100,100), xlabel="$\phi$ / (rad/m$^2$)", ylabel="Normalised")
ax.legend()

In [ ]:

minor_loop_res = clean_utils.minor_loop(
    resid_fdf_spectrum_mask=resid_fdf_spectrum_mask,
    phi_arr_radm2=phi_arr_radm2,
    phi_double_arr_radm2=phi_double_arr_radm2,
    rmsf_spectrum=rmsf_spectrum_conv,
    rmsf_fwhm=rmsf_fwhm_conv,
    max_iter=1,
    threshold=0,
    mask=0,
    gain=1
)

fig, ax = plt.subplots()
ax.plot(
    phi_arr_radm2,
    np.abs(minor_loop_res.resid_fdf_spectrum),
    "-",
    color="k",
    label="Residual",
    lw=1
)
# ax.plot(
#     phi_arr_radm2,
#     np.abs(resid_fdf_spectrum_conv),
#     "-",
#     color="k",
#     alpha=0.5,
#     lw=1,
# )